In [1]:
import gym
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle
import nltk
from nltk.tokenize import word_tokenize
#for text processing
import spacy
import re
import pandas as pd

In [2]:
env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
def fetch_pickup_drop(text = "Please book a cab from airport to hauz khaas at 3 PM"):
    li = []
    mapping = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\city.csv")
    mapp = {"dwarka sector 23" : "0",
        "dwarka sector 21" : "1",
        "hauz khaas" : "2",
        "airport" : "3"}
    for item in word_tokenize(text):
        li.append(item)
    for i in range(len(li)):
        if((li[i] == "AM") | (li[i] == "PM")):
            time_of_pickup = li[i-1] + " "+ li[i]
            li.remove(li[i-1])
            break
    li = " ".join(li)
    for key in mapp.keys():
        li = li.lower().replace(key, mapp[key])
    li = word_tokenize(li)
    found = 0
    for i in range(len(li)):
        if(li[i] == "to"):
           if li[i+1].isnumeric() :
                destination = li[i+1]
                li.remove(li[i+1])
                found = 1
                break
    if found == 0:
        for i in range(len(li)):
            if(li[i]  == "for"):
               if(li[i+1].isnumeric()):
                    destination = li[i+1]
                    li.remove(li[i+1])
                    break
    for item in li:
        if item.isnumeric():
            origin = item
    
    #Write your code here
    return( [return_location(origin), return_location(destination), time_of_pickup])
def return_location(code):
    if code == "0":
        return "dwarka sector 23"
    elif code == "1":
        return "dwarka sector 21"
    elif code == "2":
        return "hauz khaas"
    elif code == "3":
        return "airport"
    else:
        return "Theb22222222222222222222222222222"

In [4]:

env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [5]:
#Initialize Q_table
import numpy as np
#write your code here

q_table = q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [6]:
q_table.shape

(500, 6)

In [7]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output
# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 1001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")
np.save("./q_table.npy", q_table)

Episode: 1000
Training finished.

Wall time: 5.89 s


In [8]:

#Load trained q_table for evaluation
q_table = np.load("./q_table.npy")

In [17]:
def create_loc_dict(city_df):
    loc_dict = {"dwarka sector 23" : "0",\
                "dwarka sector 21" : "1",\
                "hauz khaas" : "2",\
                "airport" : "3"}
    ## Create dictionary example, loc_dict['dwarka sector 23] = 0    
    return loc_dict

In [18]:
def check_pick_up_drop_correction(pick_up, drop, line_num):
    #write your code here
    original_df = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\org_df.csv")
    original_origin = original_df['origin'][line_num]
    original_destination = original_df['dest'][line_num]
    if original_origin == pick_up and original_destination == drop:
        return True
    else:
        return False

In [ ]:
total_epochs, total_penalties = 0, 0
total_reward, wrong_predictions = 0, 0

count = 0
time_list = []
f = open(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\sms.txt", "r")
num_of_lines = 1000
city = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\city.csv")

loc_dict = create_loc_dict(city)
line_num = 0
for line in f:
    pickup, drop, _ = fetch_pickup_drop(line)
    if(check_pick_up_drop_correction(pickup, drop, line_num) == False):
        wrong_predictions +=1
    line_num += 1




episodes = 1000

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    total_reward += reward

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

In [16]:
total_epochs, total_penalties = 0, 0
total_reward, wrong_predictions = 0, 0

count = 0
time_list = []
f = open(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\sms.txt", "r")
num_of_lines = 1000
city = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\city.csv")

loc_dict = create_loc_dict(city)
line_num = 0
for line in f:
    pickup, drop, _ = fetch_pickup_drop(line)
    if(check_pick_up_drop_correction(pickup, drop, line_num) == False):
        wrong_predictions +=1
    line_num += 1

episodes = 1000
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs
    total_reward += reward

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

KeyboardInterrupt: 

In [12]:
city = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\city.csv")
print(city.shape)
city.head()

(4, 2)


,location,mapping
0,dwarka sector 23,0
1,dwarka sector 21,1
2,hauz khaas,2
3,airport,3


In [4]:
org = pd.read_csv(r"C:\Users\hp\Desktop\edureka\live session notes(given)\midterm_project2\x85973dlw\org_df.csv")
print(org.shape)
org.head()

(1000, 3)


,origin,dest,time
0,airport,hauz khaas,3 PM
1,airport,hauz khaas,6 PM
2,hauz khaas,dwarka sector 23,1 PM
3,airport,hauz khaas,1 AM
4,airport,dwarka sector 21,10 PM
